# AI Builders Week 3 Homework
## Resume matcher

In [1]:
import pymupdf
import requests

response = requests.get('https://miguel-vila.github.io/resume_EN.pdf')
response.raise_for_status()
pdf_bytes = response.content
resume = pymupdf.open(stream=pdf_bytes, filetype='pdf')

In [2]:
print("Number of pages:", len(resume))
resume_str = '\n'.join([page.get_text() for page in resume])
resume_str

Number of pages: 3


'MIGUEL VIL´A GONZ´ALEZ\nmiguel-vila @ github\nPROFESSIONAL EXPERIENCE\nSenior Software Engineer\nRemote - US East Coast Timezone\nSiriusXM\nNovember 2022 - Present\n· Member of the API tooling team, part of the platform services enablement organization.\n· Develop tooling using Smithy, an AWS DDL language, empowering teams to describe, implement, and consume\ntheir services.\n· Key projects include:\n· Developed a system for describing which Smithy services are implemented and consumed by applications,\nenabling package management and dependency tracing across the platform.\n· Implemented compatibility checks to verify service changes against potential data processing breakages.\n· Contributed to a team implementing semantic search using vector embeddings stored in OpenSearch.\n· Set up infrastructure to ingest catalog data and generate embeddings using different models.\n· Set up a client to query the different indexes using the embeddings.\nLead Software Engineer ←Senior Software En

In [3]:
import pandas as pd

jobs = pd.read_csv('jobs.csv')

jobs.head()

,id,site,job_url,job_url_direct,title,company,location,date_posted,job_type,salary_source,...,company_addresses,company_num_employees,company_revenue,company_description,skills,experience_range,company_rating,company_reviews_count,vacancy_count,work_from_home_type
0,in-764d02c34833d113,indeed,https://www.indeed.com/viewjob?jk=764d02c34833...,https://jobs.ashbyhq.com/yumaai/cdc8c768-e82b-...,AI Product Focused - Senior Fullstack / Rails ...,Yuma AI,"Boston, MA, US",2025-09-02,fulltime,direct_data,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,in-82e200b3b54e6886,indeed,https://www.indeed.com/viewjob?jk=82e200b3b54e...,https://jobs.ashbyhq.com/gallatin/b7ed9bbb-496...,DevOps Engineer,Gallatin,"El Segundo, CA, US",2025-09-02,fulltime,direct_data,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,in-c9747635cab5e167,indeed,https://www.indeed.com/viewjob?jk=c9747635cab5...,https://grnh.se/mwr1jefn2us,"Staff Software Engineer, Growth Products",Lyft,"San Francisco, CA, US",2025-09-02,NaN,direct_data,...,NaN,NaN,NaN,Multiply your earnings when you drive with Lyf...,NaN,NaN,NaN,NaN,NaN,NaN
3,in-07678b0399d77ed6,indeed,https://www.indeed.com/viewjob?jk=07678b0399d7...,https://grnh.se/e8e4gmqg2us,"Staff Software Engineer, Growth Products",Lyft,"New York, NY, US",2025-09-02,NaN,direct_data,...,NaN,NaN,NaN,Multiply your earnings when you drive with Lyf...,NaN,NaN,NaN,NaN,NaN,NaN
4,in-f89ce04ae801a7c3,indeed,https://www.indeed.com/viewjob?jk=f89ce04ae801...,https://grnh.se/nxoofj1z2us,"Staff Software Engineer, Growth Products",Lyft,"Seattle, WA, US",2025-09-02,NaN,direct_data,...,NaN,NaN,NaN,Multiply your earnings when you drive with Lyf...,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(MODEL_NAME, device=device)

embeddings = model.encode(
    jobs['description'],
    batch_size=32,
    convert_to_tensor=True,
    normalize_embeddings=True,
    show_progress_bar=False
)

jobs['embedding'] = [emb.detach().cpu().numpy() for emb in embeddings]

In [5]:
import torch.nn.functional as F

resume_embedding = model.encode(
    resume_str,
    batch_size=32,
    convert_to_tensor=True,
    normalize_embeddings=True,
    show_progress_bar=False
)

print(type(resume_embedding))
print(type(jobs['embedding'][0]))
cos_sims = [
    F.cosine_similarity(
        torch.tensor(embedding, dtype=torch.float32).unsqueeze(0),
        resume_embedding.unsqueeze(0)
    ) for embedding in jobs['embedding']
]
jobs['cos_sim'] = [cos_sim[0].detach().cpu().numpy() for cos_sim in cos_sims]
sorted_jobs = jobs.sort_values("cos_sim", ascending=False)
most_similar = sorted_jobs.head()
# print(most_similar)
# print(most_similar.shape())
for closest in most_similar.itertuples(index=False):
    print(f'Title: {closest.title}')
    # print(f'Description: {closest.description}')
    print(f'Url: {closest.job_url}')
    print(f'Similarity: {closest.cos_sim}')
print('_'*15)
most_disimilar = sorted_jobs.tail()
for closest in most_disimilar.itertuples(index=False):
    print(f'Title: {closest.title}')
    # print(f'Description: {closest.description}')
    print(f'Url: {closest.job_url}')
    print(f'Similarity: {closest.cos_sim}')
    

<class 'torch.Tensor'>
<class 'numpy.ndarray'>
Title: Senior Software Engineer, Applied AI
Url: https://www.indeed.com/viewjob?jk=a53c64fadb77ee9b
Similarity: 0.6112558245658875
Title: Software Engineer - Analytics Platforms & Experiences (APX)
Url: https://www.indeed.com/viewjob?jk=5693ba7970271f3f
Similarity: 0.6101253032684326
Title: Sr. Software Engineer II - StreamingTV
Url: https://www.indeed.com/viewjob?jk=4d149be11348a3ef
Similarity: 0.6035833358764648
Title: Software Engineer - Fullstack
Url: https://www.indeed.com/viewjob?jk=f75d0e7489b9c668
Similarity: 0.5977473258972168
Title: Developer Advocate, Developer Productivity, DevEx
Url: https://www.indeed.com/viewjob?jk=da15865a44121cbd
Similarity: 0.5840666890144348
_______________
Title: Sr Software Development Engineer
Url: https://www.indeed.com/viewjob?jk=50ebc522b8d09ed9
Similarity: 0.3583030104637146
Title: Software Engineer (7300U) - Berkeley Seismological Lab
Url: https://www.indeed.com/viewjob?jk=535907521ac63569
Simila